In [1]:
#CropCare AI: Intelligent Disease Detection and Farming Advisor

In [2]:
#Team Member 1 
#Srinivasa Reddy Julakanti

#Team Member 2
#Shaik Mohammed Hamid

#Team Member 3
#Yashwanth K

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from collections import OrderedDict
from torchvision import datasets, transforms, models

In [4]:
data_dir = 'chilli_dataset'

# Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)

In [5]:
num_classes = len(train_data.class_to_idx)
num_classes

5

In [6]:
train_data.class_to_idx

{'cercospora': 0,
 'healthy': 1,
 'mites_and_thrips': 2,
 'nutritional': 3,
 'powdery_mildew': 4}

In [7]:
model = models.resnet18(pretrained=True)
model.fc

C:\Users\srini\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\srini\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Linear(in_features=512, out_features=1000, bias=True)

In [8]:
for param in model.parameters():
    param.requires_grad = False

fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(512, 500)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(500, 5)),  # Change the number of output units to 3 for 3 classes
    ('output', nn.LogSoftmax(dim=1))
]))


model.fc = fc

In [9]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
from collections import OrderedDict
model.fc = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(512, 400)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(400, 5)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

model.to(device);

In [11]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
epochs = 10
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            
            confusion_matrix = torch.zeros(num_classes, num_classes)
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
                    for t, p in zip(labels.view(-1), top_class.view(-1)):
                            confusion_matrix[t.long(), p.long()] += 1
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            print(confusion_matrix)
            running_loss = 0
            model.train()

Epoch 1/10.. Train loss: 2.258.. Test loss: 1.866.. Test accuracy: 0.378
tensor([[59.,  1.,  2.,  0.,  0.],
        [22.,  8., 20.,  0.,  0.],
        [15.,  4., 42.,  0.,  0.],
        [40.,  4., 11.,  0.,  0.],
        [45.,  5.,  2.,  0.,  0.]])
Epoch 1/10.. Train loss: 1.693.. Test loss: 1.476.. Test accuracy: 0.354
tensor([[41.,  0.,  0.,  1., 20.],
        [ 1.,  0.,  0.,  0., 49.],
        [ 3.,  0.,  0.,  1., 57.],
        [31.,  0.,  0.,  1., 23.],
        [ 0.,  0.,  0.,  0., 52.]])
Epoch 1/10.. Train loss: 1.447.. Test loss: 1.263.. Test accuracy: 0.572
tensor([[15.,  0.,  6., 41.,  0.],
        [ 0.,  7., 26., 10.,  7.],
        [ 0.,  3., 48.,  9.,  1.],
        [ 2.,  1., 12., 40.,  0.],
        [ 2.,  0.,  1.,  1., 48.]])
Epoch 1/10.. Train loss: 1.314.. Test loss: 1.120.. Test accuracy: 0.569
tensor([[52.,  5.,  2.,  2.,  1.],
        [ 4., 16., 15.,  0., 15.],
        [ 3., 17., 34.,  3.,  4.],
        [40.,  5.,  7.,  2.,  1.],
        [ 0.,  0.,  0.,  0., 52.]])
Epoc

Epoch 5/10.. Train loss: 0.867.. Test loss: 1.448.. Test accuracy: 0.524
tensor([[34.,  0.,  2., 22.,  4.],
        [ 6.,  3., 13.,  4., 24.],
        [ 3.,  0., 22.,  6., 30.],
        [17.,  0.,  3., 32.,  3.],
        [ 0.,  0.,  0.,  0., 52.]])
Epoch 5/10.. Train loss: 0.775.. Test loss: 1.046.. Test accuracy: 0.589
tensor([[28.,  4.,  2., 28.,  0.],
        [ 2., 15., 17.,  8.,  8.],
        [ 0.,  5., 31.,  6., 19.],
        [ 6.,  3.,  5., 41.,  0.],
        [ 0.,  4.,  0.,  0., 48.]])
Epoch 5/10.. Train loss: 0.726.. Test loss: 1.256.. Test accuracy: 0.535
tensor([[35.,  0.,  6., 19.,  2.],
        [ 2.,  1., 23.,  3., 21.],
        [ 1.,  0., 30.,  5., 25.],
        [16.,  1.,  8., 28.,  2.],
        [ 0.,  0.,  0.,  0., 52.]])
Epoch 5/10.. Train loss: 0.773.. Test loss: 1.170.. Test accuracy: 0.536
tensor([[49.,  4.,  1.,  8.,  0.],
        [12., 12.,  7.,  4., 15.],
        [12.,  4., 17.,  5., 23.],
        [31.,  3.,  1., 19.,  1.],
        [ 0.,  2.,  0.,  0., 50.]])
Epoc

Epoch 8/10.. Train loss: 0.773.. Test loss: 1.340.. Test accuracy: 0.545
tensor([[26.,  1.,  3., 30.,  2.],
        [ 8.,  3., 12.,  4., 23.],
        [ 3.,  1., 23., 10., 24.],
        [ 6.,  0.,  2., 45.,  2.],
        [ 0.,  0.,  0.,  0., 52.]])
Epoch 9/10.. Train loss: 0.768.. Test loss: 1.182.. Test accuracy: 0.559
tensor([[40.,  0.,  5., 16.,  1.],
        [ 7.,  2., 15.,  5., 21.],
        [ 4.,  0., 31.,  6., 20.],
        [24.,  0.,  2., 28.,  1.],
        [ 0.,  0.,  0.,  0., 52.]])
Epoch 9/10.. Train loss: 0.771.. Test loss: 1.047.. Test accuracy: 0.580
tensor([[44.,  1.,  8.,  9.,  0.],
        [ 4.,  6., 26.,  2., 12.],
        [ 5.,  0., 38.,  6., 12.],
        [28.,  2.,  6., 19.,  0.],
        [ 0.,  0.,  0.,  0., 52.]])
Epoch 9/10.. Train loss: 0.767.. Test loss: 1.170.. Test accuracy: 0.590
tensor([[30.,  1.,  6., 25.,  0.],
        [ 1.,  4., 24.,  8., 13.],
        [ 0.,  0., 36.,  7., 18.],
        [ 7.,  1.,  7., 40.,  0.],
        [ 0.,  0.,  0.,  0., 52.]])
Epoc

In [ ]:
torch.save(model.state_dict(), 'checkpoint1.pth')

In [ ]:
def load_ckpt(ckpt_path):
    ckpt = torch.load(ckpt_path)

    model = models.resnet18(pretrained=True)
    model.fc = nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(512, 400)),
                      ('relu', nn.ReLU()),
                      ('fc2', nn.Linear(400, 5)),
                      ('output', nn.LogSoftmax(dim=1))
                      ]))

    model.load_state_dict(ckpt, strict=False)

    return model

In [ ]:
SAVE_PATH = 'checkpoint1.pth'

In [ ]:
model = load_ckpt(SAVE_PATH)

In [ ]:
import PIL
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an torch Tensor
    '''
    im = PIL.Image.open(image)
    if im.mode != 'RGB':
        im = im.convert('RGB')
    return test_transforms(im)

In [ ]:
def predict(image_path, model):
    # Predict the class of an image using a trained deep learning model.
    model.eval()
    img_pros = process_image(image_path)
    img_pros = img_pros.view(1,5,224,224)
    with torch.no_grad():
        output = model(img_pros)
    return output

In [ ]:
test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [ ]:
img_path = '.png'
log_ps = predict(img_path, model)
cls_score = int(torch.argmax(torch.exp(log_ps)))
if cls_score == 0:
    print('cercospora')
elif cls_score == 1:
    print('healthy')
elif cls_score == 2:
    print('mites_and_trips')
elif cls_score == 3:
    print('nutritional defeciency')
else:
    print('powdery mildew')
PIL.Image.open(img_path)